In [50]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
!pip install opencv-python
def save_image(file_path, file_name, save_path, save_name):
    # file_path : 원본 이미지 파일 경로
    # file_name : 원본 이미지 이름(.jpg 제외하고)
    # save_path : 신발만 자른 이미지 저장할 경로
    # save_name : 신발만 자른 이미지 저장할 이름(.jpg 제외하고)
    import numpy as np
    import matplotlib
    import cv2
    from PIL import Image
    image = cv2.imread(file_path+'/'+file_name+'.jpg')  
    # 원본 이미지 -> 나중에 잘라진 이미지 원본에서 잘라야지
    image_gray = cv2.imread(file_path+'/'+file_name+'.jpg', cv2.IMREAD_GRAYSCALE)  
    # 흑백 이미지 -> 블러 이용하기 위해
    
    b, g, r = cv2.split(image)
    image2 = cv2.merge([r, g, b])
    # b, g, r로 불러들였으니 r, g, b 순으로 재정렬
    
    # 테두리 경계 가져오기 위해 가우시안 블러 사용
    # 블러 -> 배경부분을 부드럽게 하여 내가 원하는 이미지의 경계 찾기
    # 블러 약하게 -> 원하는 부분 찾지 못함
    # 블러 강하게 -> 배경 외곽도 포함됨
    # ksize 적절히 조절 필요
    # 가우시안 필터를 이용해 이미지의 노이즈를 줄여줌.
    blur = cv2.GaussianBlur(image_gray, ksize=(3, 3), sigmaX=0)
    ret, thresh1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)
    # 기본 임계처리 (이진화 처리)
    # 임계값보다 크면 흰색, 임계값보다 작으면 검은색으로 분류하여 처리
    # cvw.threshold(src, thresh, maxval, type) -> retval, dst
    # src : input image(gray스케일 이미지)
    # thresh : 임계값
    # maxval : 임계값을 넘었을 때 적용할 value
    # type : thresholding type
    # THRESH_BINARY -> 검, 흰 딱 떨어짐
    # 다른 것들 보면 그라데이션도 있음
    # 참조 : https://opencv-python.readthedocs.io/en/latest/doc/09.imageThresholding/imageThresholding.html
    
    edged = cv2.Canny(blur, 10, 250)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    total = 0
    contours_xy = np.array(contours)
    
    # x의 min, max 찾기
    x_min, x_max = 0, 0
    value = list()
    for i in range(len(contours_xy)):
        for j in range(len(contours_xy[i])):
            value.append(contours_xy[i][j][0][0])
            x_min = min(value)
            x_max = max(value)
    # y의 min, max 찾기
    y_min, y_max = 0, 0
    value = list()
    for i in range(len(contours_xy)):
        for j in range(len(contours_xy[i])):
            value.append(contours_xy[i][j][0][1])
            y_min = min(value)
            y_max = max(value)
    # image trim 하기(불필요한 부분 잘라내기)
    x = x_min
    y = y_min
    w = x_max - x_min  # 가로 길이
    h = y_max - y_min  # 세로 길이

    # 가로 길이, 세로 길이 중 더 긴 길이를 기준으로
    # 정사각형
    img_trim = image[y_min:min(y_max, image.shape[1]), x_min:min(x_max, image.shape[0])]
    # img_trim = image[y:y+h, x:x+w]
    cv2.imwrite('org_trim.jpg', img_trim)
    org_image = cv2.imread('org_trim.jpg')
    
    # 이미지 저장
    im = Image.open('org_trim.jpg')
    # 이미지 리사이징하는 과정
    im_resize = im.resize((500, 400))
    im_resize.save(save_path+'/'+save_name+'.jpg')

In [54]:
save_image('/content/drive/MyDrive/GoogleColab/MiniProject/sample_image', 'detail_497858_65_500', '/content/drive/MyDrive/GoogleColab/MiniProject/slicing_image', 'save_3')